# 股票数据批量下载 Notebook

本notebook用于批量下载股票历史数据，不做任何分析。

## 功能
- 批量下载沪深300/中证500等指数成分股
- 下载指定股票列表
- 使用download模式（历史纯粹性）
- 自动保存到Repository

## 数据包含
- OHLCV（开高低收成交量）
- 市值数据（Yahoo Finance）
- 财务指标（AkShare）
- 复权因子

## 1. 环境设置

In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# 添加项目根目录到路径（notebook 在根目录，所以当前目录就是根目录）
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data_process.data_provider import HybridDataProvider
from data_process.repository import DataRepository

# 初始化
provider = HybridDataProvider()
repo = DataRepository("./rawdata/repository")

print("✅ 初始化完成")

✅ 初始化完成


## 2. 设置下载参数

In [2]:
# ========== 配置区域 - 根据需要修改 ==========

# 方式1: 下载指数成分股（二选一）
INDEX_CODE = '000300'  # 000300=沪深300, 000905=中证500, 000016=上证50
LIMIT = None  # 限制下载数量（None=全部下载）

# 方式2: 下载指定股票列表（如果设置了，会覆盖方式1）
SYMBOLS = ['600519', '000001', '000002']  # 例如: ['600519', '000001', '000002'] 或 None

# 日期范围
START_DATE = '2024-01-01'
END_DATE = '2026-02-20'  # 使用T-1日期（昨天）

# 其他选项
SKIP_EXISTING = True  # True=跳过已下载的股票
MODE = 'auto'  # auto=自动选择（推荐）, download=历史下载模式, update=实时更新模式

# ========== 配置结束 ==========

# 智能模式选择（如果设置为 auto）
if MODE == 'auto':
    actual_mode = provider.select_mode(END_DATE)
    print(f"🤖 智能模式选择: {actual_mode}")
else:
    actual_mode = MODE

print("📋 下载配置:")
print(f"  模式: {actual_mode} (设置: {MODE})")
print(f"  日期范围: {START_DATE} ~ {END_DATE}")
print(f"  跳过已存在: {SKIP_EXISTING}")
if SYMBOLS:
    print(f"  股票列表: {SYMBOLS}")
else:
    print(f"  指数: {INDEX_CODE}")
    print(f"  限制数量: {LIMIT if LIMIT else '全部'}")

🤖 智能模式选择: download
📋 下载配置:
  模式: download (设置: auto)
  日期范围: 2024-01-01 ~ 2026-02-20
  跳过已存在: True
  股票列表: ['600519', '000001', '000002']


## 3. 获取股票列表

In [3]:
if SYMBOLS:
    # 使用指定的股票列表
    symbols_to_download = SYMBOLS
    print(f"✅ 使用指定股票列表: {len(symbols_to_download)} 只")
else:
    # 从指数获取成分股
    print(f"正在获取 {INDEX_CODE} 成分股...")
    constituents = provider.get_index_constituents(INDEX_CODE)
    print(f"✅ 获取到 {len(constituents)} 只成分股")
    
    if LIMIT:
        symbols_to_download = constituents.index[:LIMIT].tolist()
        print(f"⚠️ 限制下载数量: {len(symbols_to_download)} 只")
    else:
        symbols_to_download = constituents.index.tolist()

print(f"\n准备下载 {len(symbols_to_download)} 只股票")
print(f"前10只: {', '.join(symbols_to_download[:10])}")

✅ 使用指定股票列表: 3 只

准备下载 3 只股票
前10只: 600519, 000001, 000002


## 4. 开始批量下载

In [4]:
print("="*80)
print("开始批量下载...")
print("="*80)
print()

results = []

for i, symbol in enumerate(symbols_to_download, 1):
    print(f"[{i}/{len(symbols_to_download)}] 下载 {symbol}...", end=" ", flush=True)
    
    try:
        # 下载数据（使用智能选择的模式）
        dataset = provider.get_complete_dataset(
            symbol,
            START_DATE,
            END_DATE,
            mode=actual_mode  # 使用智能选择的模式
        )
        
        # ========== 保存数据到Repository（符合DataRepository标准格式） ==========
        # Repository目录结构：
        # - OHLCV: data/repository/market/daily/ohlcv/{symbol}/data.csv
        # - 市值:  data/repository/fundamental/market_value/{symbol}/data.csv
        # - 财务:  data/repository/fundamental/financial_indicators/{symbol}/data.csv
        # - 复权:  data/repository/market/daily/adjustment/{symbol}/data.csv
        # 每个symbol目录自动包含: data.csv + metadata.json
        
        for key in ['ohlcv', 'market_value', 'financial', 'adjustment']:
            data = dataset.get(key)
            if data is not None and not data.empty:
                # 1. 创建副本避免修改原始数据
                data_copy = data.copy()
                
                # 2. 移除时区信息（Repository要求timezone-naive DatetimeIndex）
                if data_copy.index.tz is not None:
                    data_copy.index = data_copy.index.tz_localize(None)
                
                # 3. 使用Repository标准方法保存（自动处理格式、去重、合并、元数据）
                if key == 'ohlcv':
                    repo.save_ohlcv_data(symbol, data_copy)
                elif key == 'market_value':
                    repo.save_market_value_data(symbol, data_copy)
                elif key == 'financial':
                    repo.save_financial_data(symbol, data_copy)
                elif key == 'adjustment':
                    repo.save_adjustment_factors(symbol, data_copy)
        
        # 记录结果
        results.append({
            'symbol': symbol,
            'status': '✅ 成功',
            'ohlcv': len(dataset['ohlcv']),
            'market_value': len(dataset['market_value']),
            'financial': len(dataset['financial'])
        })
        
        print(f"✅ OHLCV({len(dataset['ohlcv'])})+ 复权因子（{len(dataset['adjustment'])}） + 市值({len(dataset['market_value'])}) + 财务({len(dataset['financial'])})")
        
    except Exception as e:
        error_msg = str(e)[:50]
        print(f"❌ 失败: {error_msg}")
        results.append({
            'symbol': symbol,
            'status': '❌ 失败',
            'ohlcv': 0,
            'market_value': 0,
            'financial': 0
        })

print()
print("="*80)
print("✅ 批量下载完成！")
print("="*80)

开始批量下载...

[1/3] 下载 600519... ✅ OHLCV(515)+ 复权因子（2357） + 市值(515) + 财务(100)
[2/3] 下载 000001... 

e:\Project\StockFrame\data_process\data_provider.py:690: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result['qfq_factor'] = result['qfq_factor'].fillna(method='ffill').fillna(method='bfill')
e:\Project\StockFrame\data_process\data_provider.py:691: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result['hfq_factor'] = result['hfq_factor'].fillna(method='ffill').fillna(method='bfill')


KeyboardInterrupt: 

## 5. 下载结果汇总

In [ ]:
# 创建结果DataFrame
results_df = pd.DataFrame(results)

# 统计
total = len(results_df)
success = len(results_df[results_df['status'] == '✅ 成功'])
failed = len(results_df[results_df['status'] == '❌ 失败'])
skipped = len(results_df[results_df['status'] == '⏭️ 跳过'])

print("📊 下载结果统计")
print("="*80)
print(f"\n总计: {total} 只股票")
print(f"成功: {success} 只 ({success/total*100:.1f}%)")
print(f"失败: {failed} 只 ({failed/total*100:.1f}%)")
print(f"跳过: {skipped} 只 ({skipped/total*100:.1f}%)")

# 成功下载的数据统计
if success > 0:
    success_df = results_df[results_df['status'] == '✅ 成功']
    print(f"\n📈 平均数据量（成功下载）:")
    print(f"  OHLCV: {success_df['ohlcv'].mean():.0f} 条")
    print(f"  市值: {success_df['market_value'].mean():.0f} 条")
    print(f"  财务: {success_df['financial'].mean():.0f} 条")

print("\n详细结果:")
print("="*80)
display(results_df)

## 6. 失败股票列表（如有）

In [ ]:
failed_df = results_df[results_df['status'] == '❌ 失败']

if len(failed_df) > 0:
    print(f"⚠️ 以下 {len(failed_df)} 只股票下载失败:")
    print("="*80)
    for symbol in failed_df['symbol']:
        print(f"  - {symbol}")
    print("\n💡 建议: 可以单独重试这些股票")
else:
    print("✅ 所有股票下载成功！")

## 7. 导出结果到Excel（可选）

In [ ]:
# 导出结果
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_file = f'data/exports/download_report_{timestamp}.xlsx'

# 创建目录
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# 保存
results_df.to_excel(output_file, index=False)
print(f"✅ 结果已导出到: {output_file}")

## 8. 数据存储位置

In [ ]:
print("📂 数据存储位置:")
print("="*80)
print(f"\n项目根目录: {project_root}")
print(f"\n✅ DataRepository标准目录结构:")
print("data/")
print("└── repository/")
print("    ├── market/")
print("    │   └── daily/")
print("    │       ├── ohlcv/           ← OHLCV数据")
print("    │       │   └── {symbol}/")
print("    │       │       ├── data.csv")
print("    │       │       └── metadata.json")
print("    │       └── adjustment/      ← 复权因子")
print("    │           └── {symbol}/")
print("    │               ├── data.csv")
print("    │               └── metadata.json")
print("    └── fundamental/")
print("        ├── market_value/        ← 市值数据")
print("        │   └── {symbol}/")
print("        │       ├── data.csv")
print("        │       └── metadata.json")
print("        └── financial_indicators/ ← 财务数据")
print("            └── {symbol}/")
print("                ├── data.csv")
print("                └── metadata.json")

# 检查数据目录
repo_dir = os.path.join(project_root, 'data', 'repository', 'market', 'daily', 'ohlcv')
if os.path.exists(repo_dir):
    stock_count = len([d for d in os.listdir(repo_dir) if os.path.isdir(os.path.join(repo_dir, d))])
    print(f"\n✅ 当前已下载 {stock_count} 只股票的数据")
    
    # 验证数据格式
    if stock_count > 0:
        # 检查第一个股票的格式
        sample_symbol = [d for d in os.listdir(repo_dir) if os.path.isdir(os.path.join(repo_dir, d))][0]
        sample_dir = os.path.join(repo_dir, sample_symbol)
        has_data_csv = os.path.exists(os.path.join(sample_dir, 'data.csv'))
        has_metadata = os.path.exists(os.path.join(sample_dir, 'metadata.json'))
        
        print(f"\n🔍 格式验证（示例: {sample_symbol}）:")
        print(f"  - data.csv存在: {'✅' if has_data_csv else '❌'}")
        print(f"  - metadata.json存在: {'✅' if has_metadata else '❌'}")
        
        if has_data_csv and has_metadata:
            print(f"  - 格式符合Repository标准: ✅")
else:
    print(f"\n⚠️ 数据目录不存在: {repo_dir}")
    print("提示: 首次运行时会自动创建目录结构")

---

## 总结

### ✅ 完成的工作
- 批量下载股票历史数据
- 数据自动保存到Repository
- 生成下载结果报告

### 📊 数据说明
- **模式**: auto（智能选择模式）
- **覆盖率**: 根据 end_date 自动选择 download 或 update
- **数据源**: Yahoo Finance（市值） + AkShare（OHLCV、财务）

### 💡 下一步
1. 使用 `data_analysis_example.ipynb` 进行数据分析
2. 每日使用update模式更新最新数据
3. 开始因子挖掘和回测

### 📚 相关文档
- [智能模式选择](docs/AUTO_MODE_SELECTION.md)
- [智能模式选择示例](example_auto_mode_selection.py)

---

**状态**: ✅ 下载完成